In [1]:
from humemai.utils import disable_logger
disable_logger()

from gremlin_python.structure.graph import Graph, Vertex, Edge
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.driver.serializer import GraphSONSerializersV3d0
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.traversal import P, T, Direction

import json
from humemai.janusgraph import Humemai
from tqdm.auto import tqdm

humemai = Humemai()
humemai.start_containers()
humemai.connect()
# humemai.remove_all_data()


with open("./raw-data.json") as f:
    data = json.load(f)

# humemai.disconnect()
# humemai.stop_containers()
# humemai.remove_containers()

from humemai.utils import disable_logger
disable_logger()

/home/tk/.virtualenvs/humemai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
DEBUG:docker.utils.config:Trying paths: ['/home/tk/.docker/config.json', '/home/tk/.dockercfg']
DEBUG:docker.utils.config:No config file found
DEBUG:docker.utils.config:Trying paths: ['/home/tk/.docker/config.json', '/home/tk/.dockercfg']
DEBUG:docker.utils.config:No config file found
DEBUG:urllib3.connectionpool:http://localhost:None "GET /version HTTP/11" 200 834
DEBUG:docker.utils.config:Trying paths: ['/home/tk/.docker/config.json', '/home/tk/.dockercfg']
DEBUG:docker.utils.config:No config file found
DEBUG:docker.utils.config:Trying paths: ['/home/tk/.docker/config.json', '/home/tk/.dockercfg']
DEBUG:docker.utils.config:No config file found
DEBUG:urllib3.connectionpool:http://localhost:None "GET /version HTTP/11" 200 834


In [ ]:
list_of_labels = ["Amsterdam", "Taewoon"]

vertices = (
    humemai.g.V()
    .hasLabel(*list_of_labels)  # Filter by vertex labels
    .inE("has_episodic_memory", "has_semantic_memory")  # Check for incoming edges with specific labels
    .inV()  # Traverse back to the vertices
    .toList()  # Convert to a list
)
[vertice.id for vertice in vertices]

In [ ]:
from gremlin_python.process.traversal import P

# Define the time range
start_time = "2024-12-01T00:00:00"
end_time = "2024-12-03T23:59:59"

# Use P.and_() instead of .and()
nodes_in_event_time_range = humemai.g.V().has("timestamp", True).hasLabel(P.gte(start_time).and_(P.lte(end_time))).out("has_episodic_memory").toList()
# native gremlin: g.V().has("timestamp", true).hasLabel(gte("2024-12-01T00:00:00").and(lte("2024-12-10T23:59:59"))).out("has_episodic_memory")

print(nodes_in_event_time_range)


In [ ]:
# Define the threshold time
threshold_time = "1999-12-03T10:16:58"

# Query for nodes with a time value greater than or equal to the threshold
nodes_with_known_since_gte = humemai.g.V().has("timestamp", True).hasLabel(P.gte(threshold_time)).out("has_semantic_memory").toList()
# native gremlin: g.V().has("timestamp", true).hasLabel(gte("1999-12-03T10:16:58")).out("has_semantic_memory")

print(nodes_with_known_since_gte)


In [ ]:
get_edges_between_vertices(humemai.g, nodes_with_known_since_gte)

In [ ]:
def is_list_of_lists_empty(list_of_lists):
    return all(not sublist for sublist in list_of_lists)


for data_point in tqdm(data[1:2]):
    label = "cp_num_" + str(data_point["cp_num"])
    properties = {
        "cp_num": data_point["cp_num"],
        "participant_num": data_point["participant"],
        "cp_name": data_point["cp_name"],
        "ticks_lasted": data_point["ticks_lasted"],
        "round_num": data_point["round_num"],
        "event_time": data_point["timestamp"],
        "remaining_time": data_point["remaining_time"],
        "remaining_rocks": data_point["remaining_rocks"],
        "victim_harm": data_point["victim_harm"],
        "num_recalled": 0,
    }
    humemai.write_long_term_vertex(label, properties)

In [ ]:
import json
from gremlin_python.structure.graph import Graph, Vertex, Edge
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.driver.serializer import GraphSONSerializersV3d0
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.traversal import P, T, Direction
from datetime import datetime

g = humemai.g

In [ ]:
alice = (
    g.addV("Alice")
    .property(
        "event_time",
        [
            "2024-12-03T10:14:58",
            "2024-12-05T10:14:58",
        ],
    )
    .next()
)

bob = (
    g.addV("Bob")
    .property(
        "event_time",
        [
            "2024-12-01T10:14:58",
            "2024-12-02T10:14:58",
        ],
    )
    .next()
)

tae = (
    g.addV("Tae")
    .property(
        "current_time",
        "2024-12-01T10:14:58",
    )
    .next()
)

g.V(alice.id).addE("knows").to(__.V(bob.id)).property(
    "event_time",
    [
        "2024-12-01T10:14:58",
        "2024-12-02T10:14:58",
        "2024-12-03T10:14:58",
        "2024-12-04T10:14:58",
        "2024-12-05T10:14:58",
    ],
).next()

In [ ]:
from humemai.janusgraph.utils.gremlin import *

alice = create_vertex(
    g,
    "Alice",
    {
        "name": "Alice",
        "age": 30,
        "event_time": [
            "2024-12-03T10:14:58",
            "2024-12-05T10:14:58",
        ],
    },
)
bob = create_vertex(
    g,
    "Bob",
    {
        "name": "Bob",
        "age": 25,
        "event_time": [
            "2024-12-01T10:14:58",
            "2024-12-02T10:14:58",
        ],
    },
)
tae = create_vertex(
    g,
    "Tae",
    {
        "name": "Tae",
        "age": 35,
        "current_time": "2024-12-01T10:14:58",
    },
)
create_edge(
    g,
    alice,
    "knows",
    bob,
    {
        "event_time": [
            "2024-12-01T10:14:58",
            "2024-12-02T10:14:58",
            "2024-12-03T10:14:58",
            "2024-12-04T10:14:58",
            "2024-12-05T10:14:58",
        ],
    },
)

In [ ]:
def query_current_time_in_range(g, start_time, end_time):
    """Query vertices and edges where `current_time` is within the range
    [start_time, end_time].

    Args:
        g: The Gremlin traversal source.
        start_time: The start of the time range (inclusive)
        end_time: The end of the time range (inclusive)

    Returns:
        A tuple (vertices, edges) containing the results.
    """
    # Query vertices
    vertices = g.V().has("current_time", P.between(start_time, end_time)).toList()

    # Query edges
    edges = g.E().has("current_time", P.between(start_time, end_time)).toList()

    return vertices, edges


# Example usage
start_time = "2024-12-01T00:00:00"
end_time = "2024-12-03T23:59:59"
vertices, edges = query_current_time_in_range(g, start_time, end_time)

# Print results
print("Vertices:")
for vertex in vertices:
    print(vertex)

print("\nEdges:")
for edge in edges:
    print(edge)

In [ ]:
def query_event_time_in_range(g, start_time, end_time):
    """Query vertices and edges where any value in `event_time` falls within [start_time, end_time].

    Args:
        g: The Gremlin traversal source.
        start_time: The start of the time range (inclusive).
        end_time: The end of the time range (inclusive).

    Returns:
        A tuple (vertices, edges) containing the results.
    """
    # Query vertices
    vertices = g.V().where(
        __.values("event_time").unfold().is_(P.between(start_time, end_time))
    ).toList()

    # Query edges
    edges = g.E().where(
        __.values("event_time").unfold().is_(P.between(start_time, end_time))
    ).toList()

    return vertices, edges


# Example usage
start_time = "2024-12-01T00:00:00"
end_time = "2024-12-03T23:59:59"
vertices, edges = query_event_time_in_range(g, start_time, end_time)

# Print results
print("Vertices:")
for vertex in vertices:
    print(vertex)

print("\nEdges:")
for edge in edges:
    print(edge)


In [ ]:
g.V().has("event_time").values("event_time").toList()[0]

In [ ]:
g.V().has("event_time").values("event_time").unfold()


In [ ]:
json.loads("[2024-12-01T10:14:58, 2024-12-02T10:14:58]")

In [ ]:
foo = Graph()

In [ ]:
mgmt = foo.openManagement()

In [ ]:
from gremlin_python.structure.graph import Graph
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection

# Connect to the Gremlin Server
graph = Graph()
g = graph.traversal().withRemote(DriverRemoteConnection('ws://localhost:8182/gremlin', 'g'))
